# Building a Travel Planner Agent with Amazon Bedrock and StrandsAgents

## Overview

This notebook demonstrates how to build a travel planning AI agent using **StrandsAgents** - an AWS open-source SDK for building AI agents with just a few lines of code. Strands Agents is an open source SDK that takes a model-driven approach to building and running AI agents in just a few lines of code. Strands scales from simple to complex agent use cases, and from local development to deployment in production.

StrandsAgents provides a simple, code-first approach to building agents that can:
- Use Amazon Bedrock models (default: Claude 3.7 Sonnet)
- Integrate with thousands of tools via StrandsAgents
- Execute complex multi-step tasks autonomously

## Agent Details

<div style="float: left; margin-right: 20px;">
    
|Feature                    |Description                                                            |
|---------------------------|-----------------------------------------------------------------------|
|**Native tools used**      |`current_time`, `calculator`                                           |
|**Custom tools created**   |`search_destination`, `search_flights`, `search_hotels`,               |
|                           |`search_activities`, `search_restaurants`, `get_weather_forecast`      |
|**Agent Structure**        |Multi-agent architecture with orchestrator and specialized agents      |
|**Model Provider**         |Amazon Bedrock (Claude 3.7 Sonnet)                                     |
|**External APIs**          |Google Search (via SerpApi) for real-time travel data                  |

</div>
<div style="clear: both;"></div>

## Agent Workflow Diagram

<div style="text-align:left">
    <img src="images/architecture.png" width="100%" />
</div>

### Architecture Overview

The travel planning agent uses a **hierarchical multi-agent architecture**:

1. **Orchestrator Agent**: The main coordinator that receives user queries and delegates to specialized agents or tools
2. **Specialized Agents**: 
   - Budget Planning Agent: Handles financial analysis and cost optimization
   - Itinerary Creation Agent: Designs day-by-day travel plans
3. **Custom Travel Tools**: Six specialized tools for searching travel information via SerpApi
4. **Native Tools**: Built-in StrandsAgents tools for calculations, time, and HTTP requests
5. **External APIs**: Integration with Google Search for real-time travel data

This architecture follows the **"Agents as Tools"** pattern, where specialized agents are wrapped as callable functions that can be invoked by the orchestrator based on the user's needs.

## What You'll Learn

By the end of this notebook, you'll be able to:
1. Set up [StrandsAgents](https://strandsagents.com/latest/) with [Amazon Bedrock](https://aws.amazon.com/bedrock/)
2. Create custom tools for travel planning
3. Integrate web search capabilities using [google-search-results](https://serpapi.com/) (SerpApi)
4. Build a multi-agent system for comprehensive trip planning

## 1. Setup and prerequisites

### Prerequisites Overview
Before we begin building our travel agent, ensure you have the following setup:

* **Python 3.10+**: Required for StrandsAgents compatibility
* **AWS Account**: With appropriate permissions for Bedrock access
* **Anthropic Claude 3.7**: Enabled on Amazon Bedrock ([setup guide](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html))
* **SerpApi API Key**: For real-time travel data ([get your key](https://serpapi.com/))

### SerpApi Setup Instructions
SerpApi provides real-time access to Google search results, which is crucial for getting current travel information:

1. Visit https://serpapi.com/dashboard
2. Sign in to your Serp API account or create one if you don't have it
3. In your dashboard, generate an API key
4. Keep this key secure - you'll need it for the environment variables

### Important: Enable Model Access in Bedrock

**Critical Step**: Before running the agent, you must enable Claude 3.7 Sonnet in your AWS Bedrock console:

1. Sign in to the AWS Management Console
2. Navigate to Amazon Bedrock service
3. Go to "Model access" in the left sidebar
4. Request access to "Claude 3.7 Sonnet"
5. Wait for approval (usually immediate for most accounts)

Without this step, your agent will fail to initialize.

### Installing Required Dependencies

Let's install all the necessary packages for our comprehensive travel planning agent:

In [ ]:
# installing pre-requisites
!pip install -r requirements.txt --quiet

### Importing dependency packages

Now let's import the dependency packages

In [ ]:
import os
import json
import random
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Optional

# SerpApi for real-time search results
from serpapi import GoogleSearch

# StrandsAgents core components
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import current_time, calculator

# Enable debug logging to see the agent's thought process and tool execution
# This is invaluable for understanding how your agent makes decisions
logging.getLogger("strands").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

### Environment Variable Configuration

Security best practice: Always use environment variables for API keys rather than hardcoding them:

In [ ]:
# Set up environment variables for API keys
# This is more secure than hardcoding in the notebook
SERPAPI_KEY = os.getenv("SERPAPI_KEY", None)
if not SERPAPI_KEY:
    print("WARNING: SERPAPI_KEY not found in environment variables")
    print("Please set it using: os.environ['SERPAPI_KEY'] = 'your_api_key'")
    # For testing, you can set it here temporarily:
    # SERPAPI_KEY = "your_actual_api_key_here"

#### Defining agent underlying LLM model

Next let's define our agent underlying model. Strands Agents natively integrate with Amazon Bedrock models, and provides the ability to configure how the model is called. Below, you can see a simple initialization of a `BedrockModel` provider, with some of the optional configurations commented out. You can learn more about configuration options, and default values, at [Strands Agents Bedrock Model Provider documentation](https://strandsagents.com/0.1.x/user-guide/concepts/model-providers/amazon-bedrock/). For our example, we will use the `Anthropic Claude 3.7 Sonnet` model from Bedrock.

In [ ]:
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    # region_name="us-east-1",
    # boto_client_config=Config(
    #    read_timeout=900,
    #    connect_timeout=900,
    #    retries=dict(max_attempts=3, mode="adaptive"),
    # ),
    # temperature=0.9,
    # max_tokens=2048,
)

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}> | initializing


## 2. Creating the Travel Search Tools

This section creates the core tools that give our agent the ability to search for real-time travel information. Each tool is designed to handle a specific aspect of travel planning and returns structured data that the agent can reason about.

### Destination Information Tool

This tool provides general travel information about any destination, giving our agent context about places to visit:

In [ ]:
@tool
def search_destination(destination: str) -> Dict:
    """
    Get general travel information about a destination.
    
    Args:
        destination: The travel destination to search for
        
    Returns:
        Dictionary containing travel guide information
    """
    params = {
        "q": f"{destination} travel guide tourism attractions",
        "api_key": SERPAPI_KEY,
        "num": 10
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    
    return {
        "destination": destination,
        "search_results": results.get("organic_results", [])[:5]
    }

### Flight Search Tool

This tool enables the agent to find current flight information and pricing:

In [ ]:
@tool
def search_flights(departure: str, destination: str, 
                  outbound_date: str, return_date: Optional[str] = None) -> Dict:
    """
    Search for flights between two locations.
    
    Args:
        departure: Departure airport code (e.g., 'LAX')
        destination: Destination airport code (e.g., 'CDG')
        outbound_date: Departure date in YYYY-MM-DD format
        return_date: Return date in YYYY-MM-DD format (optional)
        
    Returns:
        Dictionary containing flight options
    """
    params = {
        "engine": "google_flights",
        "departure_id": departure,
        "arrival_id": destination,
        "outbound_date": outbound_date,
        "currency": "USD",
        "api_key": SERPAPI_KEY
    }
    
    if return_date:
        params["return_date"] = return_date
        params["type"] = "1"  # Round trip
    else:
        params["type"] = "2"  # One way
    
    search = GoogleSearch(params)
    return search.get_dict()

### Hotel Search Tool

This tool finds accommodation options with current availability and pricing:

In [ ]:
@tool
def search_hotels(destination: str, check_in: str, check_out: str, 
                 adults: int = 2) -> Dict:
    """
    Search for hotels in a destination.
    
    Args:
        destination: City or location name
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        adults: Number of adults
        
    Returns:
        Dictionary containing hotel options
    """
    params = {
        "engine": "google_hotels",
        "q": destination,
        "check_in_date": check_in,
        "check_out_date": check_out,
        "adults": adults,
        "currency": "USD",
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    return search.get_dict()

### Activities and Attractions Tool

This tool discovers things to do and see at the destination:

In [ ]:
@tool
def search_activities(destination: str, interests: Optional[List[str]] = None) -> Dict:
    """
    Search for activities and attractions in a destination.
    
    Args:
        destination: City or location name
        interests: List of interests (e.g., ['museums', 'outdoor', 'food'])
        
    Returns:
        Dictionary containing activity suggestions
    """
    query = f"things to do in {destination}"
    if interests:
        query += f" {' '.join(interests)}"
    
    params = {
        "q": query,
        "api_key": SERPAPI_KEY,
        "num": 20
    }
    search = GoogleSearch(params)
    return search.get_dict()

### Restaurant Search Tool

This tool finds dining options with current information and reviews:

In [ ]:
@tool
def search_restaurants(destination: str, cuisine_type: Optional[str] = None,
                      price_range: Optional[str] = None) -> Dict:
    """
    Search for restaurants in a destination.
    
    Args:
        destination: City or location name
        cuisine_type: Type of cuisine (e.g., 'italian', 'local')
        price_range: Price range ('$', '$$', '$$$', '$$$$')
        
    Returns:
        Dictionary containing restaurant recommendations
    """
    query = f"best restaurants in {destination}"
    if cuisine_type:
        query += f" {cuisine_type}"
    if price_range:
        query += f" {price_range}"
    
    params = {
        "engine": "google_maps",
        "q": query,
        "type": "search",
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    return search.get_dict()

### Weather Forecast Tool

This tool provides weather information to help with trip planning:

In [ ]:
@tool
def get_weather_forecast(destination: str, date: Optional[str] = None) -> Dict:
    """
    Get weather forecast for a destination.
    
    Args:
        destination: City or location name
        date: Specific date for forecast (optional)
        
    Returns:
        Dictionary containing weather information
    """
    query = f"{destination} weather forecast"
    if date:
        query += f" {date}"
    
    params = {
        "q": query,
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    
    # Check for weather answer box
    weather_data = results.get("answer_box", {})
    if not weather_data:
        weather_data = results.get("organic_results", [])[:3]
    
    return {
        "destination": destination,
        "weather": weather_data
    }

## 3. Building the Travel Planning Agent

### Defining the Agent's System Prompt

The system prompt is crucial - it defines the agent's personality, capabilities, and behavior patterns:

In [ ]:
# Define the travel agent's system prompt
# This prompt establishes the agent's identity, capabilities, and operational guidelines
TRAVEL_AGENT_PROMPT = """
You are an expert travel planning assistant with access to real-time travel data through search tools.

IMPORTANT: You MUST use the available search tools to gather information. Do not rely solely on your training data.

Your goal is to help users plan amazing trips by:
1. ALWAYS using search_destination tool first to get current information about destinations
2. Using search_flights for flight information when requested
3. Using search_hotels for accommodation options
4. Using search_activities for things to do
5. Using search_restaurants for dining recommendations
6. Using get_weather_forecast for weather information
7. Using calculator tool for budget calculations, cost breakdowns, and expense planning
8. Using budget_analysis tool to analyze travel costs, compare options, 

For each query:
- Start by searching for relevant information using the tools
- Use calculator and budget_analysis tools to help users understand and plan trip costs
- Provide data-driven recommendations based on search results
- Be transparent when search results are limited or unavailable
- Create detailed itineraries with practical information

Remember: Your search tools provide real-time data that may be more current than your training data.
"""

### Creating the Primary Travel Agent

Now we instantiate our main travel planning agent with all the tools we've created:

In [ ]:
# Create the travel planning agent
# This agent serves as the primary interface for travel planning tasks
travel_agent = Agent(
    model=model,  # The Claude 3.7 Sonnet model we configured earlier
    system_prompt=TRAVEL_AGENT_PROMPT,  # Our detailed system prompt
    tools=[
        # Custom travel search tools we created
        search_destination,      # For general destination information
        search_flights,         # For flight search and pricing
        search_hotels,          # For accommodation options
        search_activities,      # For attractions and activities
        search_restaurants,     # For dining recommendations
        get_weather_forecast,   # For weather information
        
        # Built-in StrandsAgents tools for additional functionality
        calculator,             # For budget calculations and math
        current_time           # For date/time awareness
    ]
)

DEBUG | strands.tools.registry | tool_name=<search_destination> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_destination>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_flights> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_flights>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_hotels> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_hotels>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_activities> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_activities>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_restaurants> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_restauran

In [ ]:
print("✓ Travel agent created with", len(travel_agent.tool_names), "tools")

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_flights> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_hotels> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_activities> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_restaurants> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_weather_forecast> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<calculator> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<current_time> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<http_request> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<9> | tools configured


✓ Travel agent created with 9 tools


### Testing the Agent with a Simple Query

Let's test our agent with a basic calculation to ensure everything is working:

In [ ]:
# Test the agent with a simple budget calculation
# This verifies that the agent can use tools and perform basic operations
travel_agent("I usually spend about $50 per day on food. How much will meals cost for a week?")

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_flights> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_hotels> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_activities> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_restaurants> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_weather_forecast> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<calculator> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<current_time> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<http_request> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<9> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded t

I'd be happy to help you calculate your food expenses for a week based on your daily budget.

Let me use the calculator tool to figure this out for you:
Tool #26: calculator


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<1>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.tools.executor | tool_count=<1> | submitted tasks to parallel executor
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_KYpTPP-wQgq8AA-HfKZSog', 'name': 'calculator', 'input': {'expression': '50 * 7', 'mode': 'evaluate'}}> | invoking


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 50 * 7              │                                                                            │
│  │ Result    │ 350                 │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


If you spend about $50 per day on food, your total food budget for a week (7 days) would be $350.

This budget covers all your meals for the entire week. Depending on your destination, this amount might vary in terms of what it can get you. In some locations, $50 per day can cover three full meals in restaurants, while in others it might mean mixing restaurant meals with grocery shopping or street food.

Would you like me to provide more specific information about food costs for a particular destination? I can search for restaurant options and typical food costs in specific cities if

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<4>, message_count=<40> | skipping context reduction


 you'd like.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "If you spend about $50 per day on food, your total food budget for a week (7 days) would be $350.\n\nThis budget covers all your meals for the entire week. Depending on your destination, this amount might vary in terms of what it can get you. In some locations, $50 per day can cover three full meals in restaurants, while in others it might mean mixing restaurant meals with grocery shopping or street food.\n\nWould you like me to provide more specific information about food costs for a particular destination? I can search for restaurant options and typical food costs in specific cities if you'd like."}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'tooluse_KYpTPP-wQgq8AA-HfKZSog', 'name': 'calculator', 'input': {'expression': '50 * 7', 'mode': 'evaluate'}}, call_count=1, success_count=1, error_count=0, total_time=0.00793004035949707)}, cycle_dur

## 4. Creating a Multi-Agent System for Advanced Trip Planning

The "Agents as Tools" pattern allows us to create specialized agents for specific tasks and then use them as tools within a larger orchestration system. This approach provides better modularity and specialized expertise.

### Budget Planning Specialist Agent

First, let's create a specialized agent focused on budget analysis and financial planning:

In [ ]:
# System prompt for the budget planning specialist
# This agent focuses specifically on financial aspects of travel planning
BUDGET_PLANNER_PROMPT = """
You are a travel budget specialist. Your role is to:
- Calculate detailed budget breakdowns for trips
- Find cost-saving opportunities
- Suggest budget-friendly alternatives
- Provide tips for managing travel expenses
- Consider hidden costs (taxes, tips, transportation)
"""

In [ ]:
@tool
def budget_planning_agent(trip_details: str, budget_limit: Optional[float] = None) -> str:
    """
    Specialized agent for travel budget planning and optimization.
    
    Args:
        trip_details: Description of the trip including destination, duration, and preferences
        budget_limit: Maximum budget in USD (optional)
        
    Returns:
        Detailed budget analysis and recommendations
    """
    budget_agent = Agent(
        model=model,
        system_prompt=BUDGET_PLANNER_PROMPT,
        tools=[calculator, search_hotels, search_flights]
    )
    
    query = f"Create a detailed budget plan for: {trip_details}"
    if budget_limit:
        query += f" with a maximum budget of ${budget_limit}"
    
    return budget_agent(query).message

### Itinerary Creation Specialist Agent

Next, let's create an agent specialized in creating detailed, optimized itineraries:

In [ ]:
# System prompt for the itinerary creation specialist
# This agent focuses on creating optimal day-by-day travel plans
ITINERARY_CREATOR_PROMPT = """
You are an expert travel itinerary designer. Your role is to:
- Create day-by-day detailed itineraries
- Optimize travel routes and timing
- Balance activities with rest time
- Consider opening hours and peak times
- Provide alternative options for each activity
"""

In [ ]:
@tool
def itinerary_creation_agent(destination: str, duration: int, interests: List[str]) -> str:
    """
    Specialized agent for creating detailed travel itineraries.
    
    Args:
        destination: Travel destination
        duration: Number of days
        interests: List of traveler interests
        
    Returns:
        Detailed day-by-day itinerary
    """
    itinerary_agent = Agent(
        model=model,
        system_prompt=ITINERARY_CREATOR_PROMPT,
        tools=[search_activities, search_restaurants]
    )
    
    query = f"Create a {duration}-day itinerary for {destination} focusing on: {', '.join(interests)}"
    return itinerary_agent(query).message

### Master Orchestrator Agent

Finally, let's create the orchestrator agent that coordinates all specialized agents and tools:

In [ ]:
# System prompt for the master orchestrator agent
# This agent serves as the intelligent coordinator for complex travel planning tasks
ORCHESTRATOR_PROMPT = """
You are the lead travel planning coordinator. You delegate tasks to specialized agents:
- For budget analysis → Use budget_planning_agent
- For detailed itineraries → Use itinerary_creation_agent
- For general travel information → Use the search tools directly

Coordinate these resources to create comprehensive travel plans that address all aspects
of the user's trip. Always ensure the final plan is cohesive and practical.
"""

In [ ]:
# Create the master orchestrator agent with access to all tools and specialist agents
orchestrator = Agent(
    model=model,
    system_prompt=ORCHESTRATOR_PROMPT,
    tools=[
        # Specialized agent tools (agents as tools pattern)
        budget_planning_agent,      # For comprehensive budget analysis
        itinerary_creation_agent,   # For detailed itinerary creation
        
        # Direct search tools for immediate information needs
        search_destination,         # For general destination research
        search_flights,            # For flight information
        search_hotels,             # For accommodation research
        get_weather_forecast       # For weather planning
    ]
)

DEBUG | strands.tools.registry | tool_name=<budget_planning_agent> | registering function tool
DEBUG | strands.tools.registry | tool_name=<budget_planning_agent>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<itinerary_creation_agent> | registering function tool
DEBUG | strands.tools.registry | tool_name=<itinerary_creation_agent>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_destination> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_destination>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_flights> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_flights>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_hotels> | registering function tool
DEBUG | strands.tools.registry | to

## 5. Testing the Travel Planning Agent

### Example 1: Simple Destination Query

Let's start with a straightforward query about visiting a specific destination:

In [ ]:
# Test with a simple destination query
# This demonstrates basic tool usage and destination research capabilities
response = travel_agent("Tell me about visiting Tokyo in spring. What are the must-see places?")

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_flights> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_hotels> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_activities> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_restaurants> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_weather_forecast> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<calculator> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<current_time> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<http_request> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<9> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded t

I'd be happy to tell you about visiting Tokyo in spring! Let me search for the most up-to-date information about Tokyo as a destination and its attractions during springtime.
Tool #1: search_destination


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<1>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_eDqLGoAaTFaPYxE37N6zqw', 'name': 'search_destination', 'input': {'destination': 'Tokyo'}}> | invoking
DEBUG | strands.tools.executor | tool_count=<1> | submitted tasks to parallel executor
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


Now, let me get more specific information about activities you might enjoy in Tokyo during spring:
Tool #2: search_activities


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_h1LGJVuPQfCcr1JI7CUIig', 'name': 'search_activities', 'input': {'destination': 'Tokyo', 'interests': ['cherry blossoms', 'spring festivals', 'parks', 'gardens']}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


Let me also check the weather forecast for Tokyo in spring to give you a better idea of what to expect:
Tool #3: get_weather_forecast


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_4Sw-joPCQMeZcTv0iNYZqg', 'name': 'get_weather_forecast', 'input': {'destination': 'Tokyo'}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model
DEBUG | strands.event_loop.error_handler | retry_delay_seconds=<4>, max_attempts=<6>, current_attempt=<1> | throttling exception encountered | delaying before next retry
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got r

# Visiting Tokyo in Spring: Cherry Blossoms and Must-See Destinations

Spring is one of the most magical times to visit Tokyo, primarily due to the breathtaking cherry blossom (sakura) season. Based on my latest search results, here's a comprehensive guide to help you make the most of your springtime visit to Tokyo.

## Cherry Blossom Season in Tokyo

The cherry blossom season in Tokyo typically runs from late March to mid-April, though this can vary slightly each year. This fleeting natural phenomenon, called "hanami" (flower viewing) by the Japanese, transforms the city into a spectacular pink wonderland.

### Best Time to Visit for Cherry Blossoms

According to current data, the peak bloom period usually occurs during the first two weeks of April. For 2025, forecasts suggest late March to early April will be the prime viewing time. The blossoms typically only last for about 1-2 weeks at full bloom, so timing is everything!

## Top Cherry Blossom Viewing Spots in Tokyo

Based on the 

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<8>, message_count=<40> | skipping context reduction


 your visit.

### Example 2: Complex Multi-Day Trip Planning

Now let's test the orchestrator with a complex trip planning request that requires multiple specialized agents:

In [ ]:
def get_future_date():
    """Get a random date 1-4 weeks in the future"""
    days = random.randint(7, 28)
    future = datetime.now() + timedelta(days=days)
    return future.strftime("%B %d, %Y")

In [ ]:
# Create a complex trip planning request that will engage multiple agents
trip_request = """
I want to visit Tokyo for 7 days starting {date}. 
My budget is $3000 per person excluding flights.
Please search for general travel information about Tokyo and calculate my daily budget.
""".format(date=get_future_date())

In [ ]:
# Use the orchestrator for complex planning that requires multiple specialized agents
response = orchestrator(trip_request)

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<budget_planning_agent> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<itinerary_creation_agent> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_flights> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_hotels> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_weather_forecast> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<6> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<budget_planning_agent> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<itinerary_creation_agent> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded tool config
DEBUG | strands.tools.registry | to

I'd be happy to help you plan your 7-day trip to Tokyo starting July 1, 2025! Let me start by gathering some general travel information about Tokyo and then calculate a daily budget within your $3000 per person limit (excluding flights).
Tool #20: search_destination


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<1>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.tools.executor | tool_count=<1> | submitted tasks to parallel executor
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model



Tool #21: get_weather_forecast


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_AAx18DWhR4ygWW3dJtrIlg', 'name': 'get_weather_forecast', 'input': {'destination': 'Tokyo, Japan', 'date': 'July 2025'}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model
DEBUG | strands.event_loop.error_handler | retry_delay_seconds=<4>, max_attempts=<6>, current_attempt=<1> | throttling exception encountered | delaying before next retry
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strand

...

I'd be happy to create a detailed budget plan for your 7-day trip to Tokyo, Japan. To make this more accurate, I'll need to estimate costs for various components of your trip including flights, accommodation, attractions, food, transportation, and miscellaneous expenses.

Let's start by breaking down your $3,000 budget. For Tokyo, we'll need to consider:
1. Flight costs (often a major expense)
2. Accommodation for 7 nights
3. Daily expenses for food
4. Local transportation
5. Attraction tickets and activities
6. Miscellaneous expenses (souvenirs, contingency)

Since you haven't specified your departure location, I'll need to ask:

Where will you be departing from (airport code) for your trip to Tokyo? This will help me calculate

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<2>, message_count=<40> | skipping context reduction
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


 flight costs accurately.Based on the information I've gathered, let me create a detailed budget breakdown for your 7-day Tokyo trip in July 2025:

## Tokyo Trip: July 1-7, 2025
### Budget Summary ($3,000 per person, excluding flights)

#### Weather Information:
Tokyo in July typically experiences highs of around 30°C (86°F) and lows of 23°C (73°F). It's summer season, so expect warm and humid weather with occasional rain. Pack light, breathable clothing, an umbrella, and sun protection.

#### Daily Budget Breakdown (per person):

**Accommodation (7 nights):**
- Mid-range hotel: $150-200 per night
- Total: $1,050-1,400
- *Budget-saving option: Consider business hotels (~$100/night) or hostels (~$40-60/night)*

**Daily Food:**
- Breakfast: $5-10 (convenience stores or cafes)
- Lunch: $10-15 (casual restaurants, ramen shops)
- Dinner: $20-40 (mid-range restaurants, izakayas)
- Snacks/drinks: $10-15
- Total food per day: $45-80
- Food for 7 days: $315-560

**Local Transportation:**
- 7-da

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<27>, message_count=<40> | skipping context reduction


 of your trip planning?

## 6. Adding Memory and Context Management for Personalized Experiences

To create a truly personalized travel planning experience, we need to add memory capabilities that allow the agent to remember user preferences across conversations.

### Creating a Memory Management System

In [ ]:
class TravelMemory:
    """Simple memory system for maintaining context across conversations"""
    
    def __init__(self):
        self.conversations: List[Dict] = []
        self.user_preferences: Dict = {}
    
    def add_conversation(self, user_input: str, agent_response: str):
        self.conversations.append({
            "timestamp": datetime.now().isoformat(),
            "user": user_input,
            "agent": agent_response
        })
    
    def set_preference(self, key: str, value: any):
        self.user_preferences[key] = value
    
    def get_context(self) -> str:
        context = "Previous conversations and preferences:\n"
        
        # Add user preferences
        if self.user_preferences:
            context += f"User preferences: {json.dumps(self.user_preferences)}\n"
        
        # Add recent conversations (last 3)
        recent_convos = self.conversations[-3:]
        for conv in recent_convos:
            context += f"- User: {conv['user']}\n"
            context += f"  Agent: {conv['agent'][:100]}...\n"
        
        return context

In [ ]:
# Create a travel agent with memory
memory = TravelMemory()

### Creating a Preference Management Tool

In [ ]:
@tool
def remember_preference(category: str, preference: str) -> str:
    """
    Remember user preferences for future trip planning.
    
    Args:
        category: Category of preference (e.g., 'accommodation', 'food', 'activities')
        preference: The specific preference to remember
        
    Returns:
        Confirmation message
    """
    memory.set_preference(category, preference)
    return f"I'll remember that you prefer {preference} for {category}"

### Enhanced Travel Agent with Memory

In [ ]:
# Create an enhanced travel agent that includes memory and personalization
travel_agent_with_memory = Agent(
    model=model,
    system_prompt=TRAVEL_AGENT_PROMPT + "\n\n" + memory.get_context(),  # Include personalization context
    tools=[
        # Memory and personalization tools
        remember_preference,        # For storing user preferences
        
        # All our existing travel tools
        search_destination,
        search_flights,
        search_hotels,
        search_activities,
        search_restaurants,
        get_weather_forecast,
        
        # Built-in tools
        calculator,
        current_time
    ]
)

DEBUG | strands.tools.registry | tool_name=<remember_preference> | registering function tool
DEBUG | strands.tools.registry | tool_name=<remember_preference>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_destination> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_destination>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_flights> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_flights>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_hotels> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_hotels>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<search_activities> | registering function tool
DEBUG | strands.tools.registry | tool_name=<search_activi

### Testing the Memory System

Let's test how the memory system works with user preferences:

In [ ]:
# Test the memory system by setting a preference
print("=== Testing Memory System ===")
print("Setting dietary preference...")
travel_agent_with_memory("I'm vegetarian")

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<remember_preference> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_flights> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_hotels> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_activities> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_restaurants> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_weather_forecast> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<7> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<remember_preference> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_f

I'd be happy to remember your dietary preference. This information will help me provide more tailored travel recommendations for restaurants and food experiences in the future.
Tool #23: remember_preference


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<1>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.tools.executor | tool_count=<1> | submitted tasks to parallel executor
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_JSlOgdIkRUmXTZIveHDcMw', 'name': 'remember_preference', 'input': {'category': 'food', 'preference': 'vegetarian'}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


Thank you for sharing that you're vegetarian. I've saved this preference in your profile, which will help me make more appropriate dining recommendations during your future trip planning.

When recommending restaurants or food experiences, I'll prioritize places

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<4>, message_count=<40> | skipping context reduction


 with good vegetarian options. If you have any other dietary preferences or restrictions you'd like me to know about, or if you have specific questions about vegetarian dining in particular destinations, please feel free to ask!

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Thank you for sharing that you're vegetarian. I've saved this preference in your profile, which will help me make more appropriate dining recommendations during your future trip planning.\n\nWhen recommending restaurants or food experiences, I'll prioritize places with good vegetarian options. If you have any other dietary preferences or restrictions you'd like me to know about, or if you have specific questions about vegetarian dining in particular destinations, please feel free to ask!"}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'remember_preference': ToolMetrics(tool={'toolUseId': 'tooluse_JSlOgdIkRUmXTZIveHDcMw', 'name': 'remember_preference', 'input': {'category': 'food', 'preference': 'vegetarian'}}, call_count=1, success_count=1, error_count=0, total_time=0.0024728775024414062)}, cycle_durations=[3.369424343109131], traces=[<strands.telemetry.metrics.Trace object at 0x7faa00329

DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_J8C_bMhpSOiyBhdfQq-5Qw', 'name': 'search_restaurants', 'input': {'destination': 'New York City'}}> | invoking


In [ ]:
# Now test if the agent remembers this preference in a follow-up query
print("Testing if preference is remembered...")
travel_agent_with_memory("Recommend restaurants in New York City")

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<remember_preference> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_flights> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_hotels> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_activities> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_restaurants> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_weather_forecast> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<7> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<remember_preference> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_destination> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<search_f

I'd be happy to recommend vegetarian-friendly restaurants in New York City based on your dietary preference. Let me search for some great options for you.
Tool #24: search_restaurants


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<1>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.tools.executor | tool_count=<1> | submitted tasks to parallel executor
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7faa0d44fc80>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


Based on your vegetarian preference, I've researched restaurants in New York City that offer great vegetarian options. Here are some top recommendations:

# Best Vegetarian-Friendly Restaurants in New York City

## Gramercy Tavern
This renowned New American restaurant offers excellent vegetarian options as part of their menu. They specifically list "Vegetarian options" among their offerings, and their farm-to-table approach means fresh, seasonal vegetable dishes are often highlights. Located at 42 E 20th St, they're open daily until 10:30 PM.

## Manhatta
Located on the 60th floor with stunning city views, Manhatta includes vegetarian options among their New American cuisine. The restaurant has earned a 4.7/5 rating from reviewers and offers an upscale dining experience with vegetarian dishes that don't feel like an afterthought.

## Via Carota
This rustic Italian trattoria in the West Village specifically lists vegetarian options among their offerings. Known for their seasonal Italian

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<8>, message_count=<40> | skipping context reduction


?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Based on your vegetarian preference, I\'ve researched restaurants in New York City that offer great vegetarian options. Here are some top recommendations:\n\n# Best Vegetarian-Friendly Restaurants in New York City\n\n## Gramercy Tavern\nThis renowned New American restaurant offers excellent vegetarian options as part of their menu. They specifically list "Vegetarian options" among their offerings, and their farm-to-table approach means fresh, seasonal vegetable dishes are often highlights. Located at 42 E 20th St, they\'re open daily until 10:30 PM.\n\n## Manhatta\nLocated on the 60th floor with stunning city views, Manhatta includes vegetarian options among their New American cuisine. The restaurant has earned a 4.7/5 rating from reviewers and offers an upscale dining experience with vegetarian dishes that don\'t feel like an afterthought.\n\n## Via Carota\nThis rustic Italian trattoria in the West

## Conclusion

You've now learned how to build a travel planning agent using StrandsAgents and Amazon Bedrock. This agent can:

- Search for destinations, flights, hotels, and activities in real-time
- Create detailed itineraries based on user preferences
- Manage budgets and provide cost breakdowns
- Use multiple specialized agents for complex tasks
- Stream responses for better user experience

StrandsAgents makes it remarkably simple to build sophisticated AI agents that can handle complex, multi-step tasks.

## Resources

- [StrandsAgents Documentation](https://strandsagents.com/)
- [StrandsAgents GitHub](https://github.com/strands-agents)
- [Amazon Bedrock Documentation](https://aws.amazon.com/bedrock/)
- [SerpApi Documentation](https://serpapi.com/)

Happy travels and happy coding! 🌍✈️